<a href="https://colab.research.google.com/github/erneyurrego-art/Introducci-n-a-la-IA/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Machine Learning - Entrenamiento con CatBoostClassifier

En este Colab se desarrollará todo el ciclo de vida de un modelo de machine learning: desde la inspección de los datos, pasando por la limpieza y transformación, hasta llegar al entrenamiento del modelo. Finalmente, se generará un archivo CSV para enviarlo a la competencia de Kaggle.

## Inspección de los datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, TargetEncoder

In [ ]:
try:
    d = pd.read_csv('train.csv', on_bad_lines='skip')
    display(d.head())
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
d.info()

In [ ]:
# distribución de la variable objetivo por el estracto de la vivienda

plt.figure(figsize=(10, 6))
sns.countplot(x ="F_ESTRATOVIVIENDA", hue="RENDIMIENTO_GLOBAL", data=d, palette="Paired")
plt.title("Distribución de la variable RENDIMIENTO_GLOBAL por F_ESTRATOVIVIENDA")
plt.show()

In [ ]:
# Distribución de las variables categoricas del dataset

for i, col in enumerate(d.select_dtypes(include=["object"]).columns):

  if i == 0:
    continue # la distribución de E_PRGM_ACADEMICO necesita transformación

  plt.figure(figsize=(10, 6))
  sns.countplot(data=d, x=col, order=d[col].value_counts().index, color="skyblue", edgecolor="black")
  plt.title(f"Distribución de la variable cualitativa {col}")
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
# Mapa de correlapción
plt.figure(figsize=(10, 8))
sns.heatmap(d.corr(numeric_only=True), cmap="coolwarm", center=0)
plt.title("Correlación entre las variables cuantitativas")
plt.show()

## Limpieza y transformación función

al realizar la limpieza del dataset de train y de test cree una función que se encarga de hacer la limpia total de ambos datsets

In [ ]:
print((d.isnull().mean() * 100)[d.isnull().mean() > 0])

In [ ]:
import pandas as pd
import random
from sklearn.preprocessing import OrdinalEncoder

def clean_dataset(d: pd.DataFrame) -> pd.DataFrame:

    d = d.drop(columns=["ID"], errors="ignore")

    # listado de columnas que se rellenaran con la moda
    cols_mode = [
        "E_VALORMATRICULAUNIVERSIDAD",
        "E_HORASSEMANATRABJA",
        "F_ESTRATOVIVIENDA",
        "F_EDUCACIONPADRE",
        "F_TIENELAVADORA",
        "F_TIENEAUTOMOVIL",
        "E_PAGOMATRICULAPROPIO",
        "F_TIENECOMPUTADOR",
        "F_TIENEINTERNET.1",
        "F_EDUCACIONMADRE",
        "E_HORASSEMANATRABAJA",
    ]

    for col in cols_mode:
        if col in d.columns:
            d[col] = d[col].fillna(d[col].mode()[0])

    if "F_TIENEINTERNET" in d.columns and "F_ESTRATOVIVIENDA" in d.columns:
        percentage_if = d["F_TIENEINTERNET"].value_counts(normalize=True).get("Si", 0)

        def assign_internet(row):
            if pd.notna(row["F_TIENEINTERNET"]):
                return row["F_TIENEINTERNET"]

            strata = row["F_ESTRATOVIVIENDA"]
            upper_strata = ["Estrato 3", "Estrato 4", "Estrato 5", "Estrato 6"]

            if strata in upper_strata:
                return "Si"
            else:
                return "Si" if random.random() <= percentage_if else "No"

        d["F_TIENEINTERNET"] = d.apply(assign_internet, axis=1)

    # Crear relaciones de algunas variables para mejorar la predicción del modelo
    d['RELA_PROG_VALOR_MATRICULA'] = d.apply(lambda x: f"{x['E_PRGM_ACADEMICO']}_{x['E_VALORMATRICULAUNIVERSIDAD']}", axis=1)
    d['RELA_PROG_VALOR_MATRICULA_DEPARTAMENTO'] = d.apply(lambda x: f"{x['E_PRGM_ACADEMICO']}_{x['E_VALORMATRICULAUNIVERSIDAD']}_{x['E_PRGM_DEPARTAMENTO']}", axis=1)
    d['RELA_PROG_DEPARTAMENTO'] = d.apply(lambda x: f"{x['E_PRGM_ACADEMICO']}_{x['E_PRGM_DEPARTAMENTO']}", axis=1)



    # hacer conversion de la variable objetivo
    if 'RENDIMIENTO_GLOBAL' in d.columns:
      try:
          orden_rendimiento = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
          d['RENDIMIENTO_GLOBAL'] = d['RENDIMIENTO_GLOBAL'].replace(orden_rendimiento)
      except Exception as e:
          print("Ocurrío un error al transformar la variable RENDIMIENTO_GLOBAL")

    return d

## Entrenamiento y construcción del modelo de machine learning con el algoritmo de HistGradientBoosting

1. Creación del modelo.
2. Entrenamiento.
3. Predicción.
4. Accurracy and score del modelo con test.
5. CSV solución para Kaggle.

In [ ]:
!pip install catboost

In [ ]:
d = clean_dataset(d) # limpiar y transformar el dataset con función anterior.
d.head(5)

In [ ]:
d.columns

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = d.drop('RENDIMIENTO_GLOBAL', axis=1)
y = d['RENDIMIENTO_GLOBAL']

categorical_features = X.select_dtypes(include=["object"]).columns.tolist()
xtr, xts, ytr, yts = train_test_split(X, y, test_size=0.2, random_state=16, stratify=y)

print("Tamaño del dataset original: ", d.shape)
print("Tamaño del dataset de entrenamiento: ", xtr.shape)
print("Tamaño del dataset de test: ", xts.shape)

In [ ]:
print((d.isnull().mean() * 100)[d.isnull().mean() > 0])

In [ ]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.07,
    depth=6,
    l2_leaf_reg=3,
    bootstrap_type='Bernoulli',
    subsample=0.75,
    auto_class_weights='Balanced',
    thread_count=4,
    verbose=100,
    task_type='CPU',
    eval_metric='Accuracy',
)

model.fit(xtr, ytr, eval_set=(xts, yts), cat_features=categorical_features, use_best_model=True)

# Predicción del dataset de test
y_pred = model.predict(xts)

# Resultados del modelo
accuracy = accuracy_score(yts, y_pred)
print(f"Precisión del modelo: {accuracy}")

0:	learn: 0.4028953	test: 0.4062775	best: 0.4062775 (0)	total: 6.44s	remaining: 1h 47m 14s
100:	learn: 0.4381214	test: 0.4392188	best: 0.4393270 (99)	total: 9m 25s	remaining: 1h 23m 51s
200:	learn: 0.4428412	test: 0.4433121	best: 0.4433985 (197)	total: 19m 31s	remaining: 1h 17m 37s
300:	learn: 0.4455128	test: 0.4444632	best: 0.4446675 (269)	total: 29m 53s	remaining: 1h 9m 25s
400:	learn: 0.4471178	test: 0.4449247	best: 0.4451638 (380)	total: 39m 51s	remaining: 59m 32s
500:	learn: 0.4484945	test: 0.4450684	best: 0.4451638 (380)	total: 49m 58s	remaining: 49m 46s


### Predicción para la competencia con el test

Luego de completar la etapa de entrenamiento, pasamos a la parte de predecir el dataset de test para subirlo a la competencia de Kaggle.

In [ ]:
test = pd.read_csv("test.csv", on_bad_lines="skip")

test_ids = test["ID"]
clean_test = clean_dataset(test)

pred_final = model.predict(clean_test)

In [ ]:
mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
reverse_mapping = {v: k for k, v in mapping.items()}
final_predictions = [reverse_mapping[p[0]] for p in pred_final]

In [ ]:
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": final_predictions
})

submission

In [ ]:
submission.to_csv("submission.csv", index=False)